# Part I:

**Please make sure of the following while using the program:**
* Run all the cells proir to 'The Program Cell' each time you want to use the program.
* The buttons for the statistical calculations (like the ensemble mean plotting...etc) must be clicked in order from left to right to ensure that you'll not get errors in use.
* When you enter the desired number of a sample function, make sure it's decremented by 1(i.e. if you want the 6th sample enter 5)

**Importing necessary libraries:**

In [ ]:
import numpy as np
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import matplotlib.image as img
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from tkinter import messagebox
from mpl_toolkits import mplot3d
import statistics as st

**Declaring global variables and creating different functions to make the program work:**

In [ ]:
global ensemble_no_entry
global ensemble_entry
global scrollable_frame
global the_ENSEMBLE
global row_nxt 
global sample_len
global nth_samp_entry
global ij_acf_entry
global time_acf_entry
global i_j
global time_vec
global Time_step
global Time_vector
global plot_ens_entry
global the_PSD
global acf

row_nxt = 16
the_ENSEMBLE = list() 

#===============================================================================================================================
#This function plots the ensemble mean of the random process whenever the 'Calculate and PLot Ensemble Mean' button is clicked.
def calcPlotEnsMean(event = None):
        ensemble_mean = list()
        
        for i in range(sample_len):
            var = 0
            for j in range(int(ensemble_no_entry.get())):
                var = var + the_ENSEMBLE[j][i]
            ensemble_mean.append(var)

        ensemble_mean = (1/int(ensemble_no_entry.get())) * np.array(ensemble_mean)

        
        plt_ens_mean = tk.Tk()
        plt_ens_mean.title("The Ensemble Mean Plot of the Process")
        figure2 = plt.Figure(figsize=(6,5), dpi=100)
        ax2 = figure2.add_subplot(111)
        ax2.plot(Time_vector, ensemble_mean)
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Amplitude')    
        canvas2 = FigureCanvasTkAgg(figure2, master = plt_ens_mean)   
        canvas2.draw() 
        canvas2.get_tk_widget().pack()
    
        
        
#===============================================================================================================================
#As soon as the 'Calculate and Plot PSD' button is clicked, this function gets the PSD of the process plotted.        
def calcPlotPSD(event = None):
        global the_PSD
        the_PSD = np.round(np.abs(np.fft.fft(acf)))
        
        g=list()

        for i in range(len(acf)):
            g.append(st.mean([row[i] for row in the_PSD]))

        plt_psd = tk.Tk()
        plt_psd.title("The PSD Plot of the Process")
        figure2 = plt.Figure(figsize=(6,5), dpi=100)
        ax2 = figure2.add_subplot(111)
        ax2.plot(range(len(the_PSD)), g)
        ax2.set_xlabel('Frequency Bins')
        ax2.set_ylabel('Amplitude')    
        canvas2 = FigureCanvasTkAgg(figure2, master = plt_psd)   
        canvas2.draw() 
        canvas2.get_tk_widget().pack()        
                
#===============================================================================================================================
#This function calculates the ACF between the ith sample and the jth sample of the process, displays the result in a messagebox
#and displays the entry of n for the nth sample for time ACF calculation. 
def calcijACF():
        global time_acf_entry
        i_and_j = np.array(ij_acf_entry.get().split(","))
        i_j = list()
        
        for i in range(len(i_and_j)):
            i_j.append(int(i_and_j[i])) 
        
        sample_i = list()
        sample_j = list()
        
        for i in range(int(ensemble_no_entry.get())):
                sample_i.append(the_ENSEMBLE[i][i_j[0]])
                sample_j.append(the_ENSEMBLE[i][i_j[1]])
        
        ACF_result = 0 
        
        for i in range(int(ensemble_no_entry.get())):
            ACF_result = ACF_result + sample_i[i]*sample_j[i]
        
        ACF_avg = ACF_result/int(ensemble_no_entry.get())
        messagebox.showinfo("ACF Between Samples i and j",f"The ACF between the samples is {ACF_avg}.")
    
        
        #Calculate time ACF of nth sample.
        ttk.Label(scrollable_frame, text = "Enter the sample No. to caculate the Time ACF:",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 21+int(ensemble_no_entry.get()), padx = 10, pady = 25) 

        n = tk.StringVar()  
        time_acf_entry = tk.Entry(scrollable_frame, textvariable = n)
        time_acf_entry.grid(row = 21+int(ensemble_no_entry.get()), column = 1)

        time_acf_button =  tk.Button(scrollable_frame, text = 'Calculate Time \n ACF', command = calcTimeACF)
        time_acf_button.grid(row = 21+int(ensemble_no_entry.get()), column = 3)
            
        
        
#===============================================================================================================================
#This function calculates the time average of the nth sample and displays a label, entry and button for the sake of calculating
#the ACF between the ith and jth samples of the process.
def calcTA(event = None):
                    
        global ij_acf_entry

        nth = int(nth_samp_entry.get())
        nth_samp = the_ENSEMBLE[nth]
        time_avg = 0
        
        for i in range(sample_len):
            time_avg = time_avg + nth_samp[i]
        
        time_avg = Time_step*time_avg*(1/sample_len)
        messagebox.showinfo("Time Average",f"The Time average of {nth} the sample is {time_avg}.")
                    
        #Calculate ACF between ith and jth samples of the process
        ttk.Label(scrollable_frame, text = "Enter i & j respectively separated by ',' to calculate the ACF [0 to M-1]:",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 20+int(ensemble_no_entry.get()), padx = 10, pady = 25) 

        n = tk.StringVar()  
        ij_acf_entry = tk.Entry(scrollable_frame, textvariable = n)
        ij_acf_entry.grid(row = 20+int(ensemble_no_entry.get()), column = 1)

        acf_button =  tk.Button(scrollable_frame, text = 'Enter i and j', command = calcijACF)
        acf_button.grid(row = 20+int(ensemble_no_entry.get()), column = 3)
#===============================================================================================================================
#This function calculates and plot the time ACF of the nth sample whare n is entered by the user.
def calcTimeACF():
        nth_sample_tacf  = int(time_acf_entry.get())
        nth_samplee = the_ENSEMBLE[nth_sample_tacf]
        xxx = list()
        for j in range(1,len(nth_samplee)-1):
            xx = list()
            for i in range(len(nth_samplee)-j):
                xx.append(nth_samplee[i]*nth_samplee[i+j])

            n= np.sum(xx)/len(xx)
            xxx.append(n)
        #plot the time ACF
        plt_tacf = tk.Tk()
        plt_tacf.title("The Time ACF of a Sample Function")
        figure2 = plt.Figure(figsize=(6,5), dpi=100)
        ax2 = figure2.add_subplot(111)
        ax2.plot(range(1,len(nth_samplee)-1), xxx)
        ax2.set_xlabel('Time Difference')
        ax2.set_ylabel('Amplitude')    
        canvas2 = FigureCanvasTkAgg(figure2, master = plt_tacf)   
        canvas2.draw() 
        canvas2.get_tk_widget().pack()
#===============================================================================================================================

def totAvgPow():
        messagebox.showinfo("Total Average Power of the Process",f"The total average power of the process is {np.sum(the_PSD)/len(the_PSD)}")

#===============================================================================================================================
#This Function calculates the ACF between ith and jth samples, and it's called each time a different combination of i and j 
#is created in the 3D AFC plotting function below.
def acfij(_i,_j):
            samp_i = list()
            samp_j = list()
            
            for i in range(len(the_ENSEMBLE)):
                    samp_i.append(the_ENSEMBLE[i][_i])
                    samp_j.append(the_ENSEMBLE[i][_j])
            
            ACF_result = 0 
            
            for i in range(len(the_ENSEMBLE)):
                ACF_result = ACF_result + samp_i[i]*samp_j[i]
            
            ACF_avg = ACF_result/len(the_ENSEMBLE)
            return ACF_avg
#This function generates a 3D ACF plot.     
def pltACF3D():
            global acf   
            def f(i, j):
                return acfij(i,j)

            x = range(len(the_ENSEMBLE[0]))
            y = range(len(the_ENSEMBLE[0]))

            X, Y = np.meshgrid(x, y)
            Z = f(X, Y)
            acf = f(X, Y)
            
            three_d_rt = tk.Tk()
            three_d_rt.title("The 3-D ACF Plot")
            figure2 = plt.Figure(figsize=(6,5), dpi=100)
            ax2 = figure2.add_subplot(111, projection = "3d")
            ax2.scatter(X, Y, Z)
            canvas2 = FigureCanvasTkAgg(figure2, master = three_d_rt)   
            canvas2.draw() 
            canvas2.get_tk_widget().pack()

#===============================================================================================================================
#This function displays buttons for ensemble mean, PSD and average power calculation and plotting, and entry of n for the nth sample
#for time average calcuation.       
def showButtons():
            global nth_samp_entry
 

            #click this button to get the mean of the ensembles.
            ens_mean_button =  tk.Button(scrollable_frame, text = 'Calculate and Plot \n Ensemble Mean', command = calcPlotEnsMean)
            ens_mean_button.grid(row = 18+int(ensemble_no_entry.get()), column = 0)

            #click this button to get the PSD of the process
            PSD_button =  tk.Button(scrollable_frame, text = 'Calculate and Plot \n the PSD', command = calcPlotPSD)
            PSD_button.grid(row = 18+int(ensemble_no_entry.get()), column = 2)

            #click this button to get the total average power of the process.
            tot_avg_pow_button =  tk.Button(scrollable_frame, text = 'Calculate Total \n Average Power', command = totAvgPow)
            tot_avg_pow_button.grid(row = 18+int(ensemble_no_entry.get()), column = 3) 
            
            #click this button to get 3D ACF plot.
            plt_acf_button =  tk.Button(scrollable_frame, text = 'Plot ACF', command = pltACF3D)
            plt_acf_button.grid(row = 18+int(ensemble_no_entry.get()), column = 1) 

            #Calculate time average for nth sample
            ttk.Label(scrollable_frame, text = "Enter the sample No. to caculate the Time Average [0 to M-1]:",font = ("Times New Roman", 10)).grid(column = 0,  
            row = 19+int(ensemble_no_entry.get()), padx = 10, pady = 25) 

            sss = tk.StringVar()  
            nth_samp_entry = tk.Entry(scrollable_frame, textvariable = sss)
            nth_samp_entry.grid(row = 19+int(ensemble_no_entry.get()), column = 1)
            

            time_avg_button =  tk.Button(scrollable_frame, text = 'Calculate Time \n Average', command = calcTA)
            time_avg_button.grid(row = 19+int(ensemble_no_entry.get()), column = 3)


#===============================================================================================================================
#This Function get M sample functions out of the sample functions entered by the user plotted, where M is entered by user.    
def plotEns(event = None):
        root2 = tk.Tk()
        root2.title("Sample Function Plots")
        container2 = ttk.Frame(root2)
        canvasss = tk.Canvas(container2, width = 730, height = 500)

        scrollbar2 = ttk.Scrollbar(container2, orient="vertical", command=canvasss.yview)
        scrollable_frame2 = ttk.Frame(canvasss)

        scrollable_frame2.bind(
            "<Configure>",
            lambda e: canvasss.configure(
                scrollregion=canvasss.bbox("all")
            )
        )

        canvasss.create_window((1, 1), window=scrollable_frame2, anchor="nw")
        canvasss.configure(yscrollcommand=scrollbar2.set)
        showButtons()

        for i in range(int(plot_ens_entry.get())):

                        figure2 = plt.Figure(figsize=(6,5), dpi=100)
                        ax2 = figure2.add_subplot(111)
                        ax2.plot(Time_vector, the_ENSEMBLE[i])
                        ax2.set_xlabel('Time')
                        ax2.set_ylabel('Amplitude')    
                        canvas2 = FigureCanvasTkAgg(figure2, master = scrollable_frame2)   
                        canvas2.draw() 
                        canvas2.get_tk_widget().pack()

        container2.pack()
        canvasss.pack(side="left", fill="both", expand=True)
        scrollbar2.pack(side="right", fill="y")

        
#===============================================================================================================================
#This function enables the user to enter the number of sample functions to be plotted.
def entrNoPltEnsemble(event = None):
        global Time_vector, plot_ens_entry, Time_step

        Time_vector = time_vec.get().split(",")
        
        for i in range(len(Time_vector)):
            Time_vector[i] = float(Time_vector[i])
        
        Time_step = Time_vector[1] - Time_vector[0]
        
        ttk.Label(scrollable_frame, text = "Enter the number of sample function you want to plot:",font = ("Times New Roman", 10)).grid(column = 0,  
        row = row_nxt+2, padx = 10, pady = 25) 

        n = tk.StringVar()  
        plot_ens_entry = tk.Entry(scrollable_frame, textvariable = n)
        plot_ens_entry.grid(row = row_nxt+2, column = 1)

        plot_ens_button =  tk.Button(scrollable_frame, text = 'View Ensembles', command = plotEns)
        plot_ens_button.grid(row = row_nxt+2, column = 3)

        
#===============================================================================================================================        

def haveListofLists(event = None):
        global the_ENSEMBLE
        global row_nxt 
        global sample_len
        global time_vec
        
        sam_func = ensemble_entry.get()
        sample_func = sam_func.split(",")
        sample_len = len(sample_func)
        ensemble = np.zeros(len(sample_func), dtype = float)
        
        for i in range(len(sample_func)):
            ensemble[i] = float(sample_func[i])
            
        the_ENSEMBLE.append(ensemble)
                
        if row_nxt - 15 != int(ensemble_no_entry.get()):
            row_nxt = row_nxt + 1
            takeEnsembles()
        
        else:
            global time_vec
            ttk.Label(scrollable_frame, text = "Enter the time vector:",font = ("Times New Roman", 10)).grid(column = 0,  
            row = row_nxt+1, padx = 10, pady = 25) 

            sss = tk.StringVar()  
            time_vec = tk.Entry(scrollable_frame, textvariable = sss)
            time_vec.grid(row = row_nxt+1, column = 1)

            time_vec_button =  tk.Button(scrollable_frame, text = 'Enter Vector', command = entrNoPltEnsemble)
            time_vec_button.grid(row = row_nxt+1, column = 3)



            
#===============================================================================================================================                  
#This function allows the user to enter the sample function of the process as long as the number of sample functions entered by the user doesn't
#equal the number of sample function entered by the user at the beginning     
def takeEnsembles(event = None):
            global sam_func 
            global ensemble_entry
            global row_nxt 

            ttk.Label(scrollable_frame, text = "Enter a sample function of the process [separate values by commas]:",font = ("Times New Roman", 10)).grid(column = 0,  
            row = row_nxt, padx = 10, pady = 25) 

            n = tk.StringVar()  
            ensemble_entry = tk.Entry(scrollable_frame, textvariable = n)
            ensemble_entry.grid(row = row_nxt, column = 1)


            enseb_ent = tk.Button(scrollable_frame, text = 'Enter Sample \n Function', command = haveListofLists)
            enseb_ent.grid(row = row_nxt, column = 3)


**The Program Cell:**

In [ ]:
#look up the reference for the code that I used to create a scrollable frame. 
root = tk.Tk()
root.title("Stochastic Process Statistics Calculator")
container = ttk.Frame(root)
canvas = tk.Canvas(container, width = 730, height = 500)

scrollbar = ttk.Scrollbar(container, orient="vertical", command=canvas.yview)
scrollable_frame = ttk.Frame(canvas)

scrollable_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(
        scrollregion=canvas.bbox("all")
    )
)

canvas.create_window((1, 1), window=scrollable_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)

#A guiding label for the user to enter the number of emsembles.
ttk.Label(scrollable_frame, text = "Enter the number of the process ensembles:",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 15, padx = 10, pady = 25) 

#An entry to enable the user to enter the number of ensembles (M).
n = tk.StringVar()  
ensemble_no_entry = tk.Entry(scrollable_frame, textvariable = n)
ensemble_no_entry.grid(row = 15, column = 1)

#A button to display entries that enables the user to get the amplitudes of a single ensemble a number M of times. 
button_1 = tk.Button(scrollable_frame, text = 'Enter', command = takeEnsembles)
button_1.grid(row = 15, column = 3) 




container.pack()
canvas.pack(side="left", fill="both", expand=True)
scrollbar.pack(side="right", fill="y")

root.mainloop()

**Generate Cosines:**

In [ ]:
time_enter = np.arange(-5,5,0.01)
cos1 = np.round(5*np.cos((2*np.pi*time_enter)+(np.pi/7)),2)
cos1

array([ 4.5 ,  4.36,  4.2 ,  4.02,  3.82,  3.61,  3.39,  3.15,  2.9 ,
        2.64,  2.37,  2.09,  1.8 ,  1.5 ,  1.2 ,  0.89,  0.58,  0.27,
       -0.04, -0.36, -0.67, -0.98, -1.29, -1.59, -1.88, -2.17, -2.45,
       -2.72, -2.98, -3.22, -3.46, -3.68, -3.88, -4.07, -4.25, -4.4 ,
       -4.54, -4.67, -4.77, -4.85, -4.92, -4.97, -4.99, -5.  , -4.99,
       -4.95, -4.9 , -4.83, -4.74, -4.63, -4.5 , -4.36, -4.2 , -4.02,
       -3.82, -3.61, -3.39, -3.15, -2.9 , -2.64, -2.37, -2.09, -1.8 ,
       -1.5 , -1.2 , -0.89, -0.58, -0.27,  0.04,  0.36,  0.67,  0.98,
        1.29,  1.59,  1.88,  2.17,  2.45,  2.72,  2.98,  3.22,  3.46,
        3.68,  3.88,  4.07,  4.25,  4.4 ,  4.54,  4.67,  4.77,  4.85,
        4.92,  4.97,  4.99,  5.  ,  4.99,  4.95,  4.9 ,  4.83,  4.74,
        4.63,  4.5 ,  4.36,  4.2 ,  4.02,  3.82,  3.61,  3.39,  3.15,
        2.9 ,  2.64,  2.37,  2.09,  1.8 ,  1.5 ,  1.2 ,  0.89,  0.58,
        0.27, -0.04, -0.36, -0.67, -0.98, -1.29, -1.59, -1.88, -2.17,
       -2.45, -2.72,

In [ ]:
cos2 = np.round(5*np.cos((2*np.pi*time_enter)+(np.pi/6)),2)
cos2

array([ 4.33,  4.16,  3.98,  3.78,  3.57,  3.35,  3.11,  2.85,  2.59,
        2.32,  2.03,  1.74,  1.45,  1.14,  0.83,  0.52,  0.21, -0.1 ,
       -0.42, -0.73, -1.04, -1.34, -1.64, -1.94, -2.22, -2.5 , -2.77,
       -3.02, -3.27, -3.5 , -3.72, -3.92, -4.11, -4.28, -4.43, -4.57,
       -4.69, -4.79, -4.87, -4.93, -4.97, -5.  , -5.  , -4.98, -4.95,
       -4.89, -4.82, -4.72, -4.61, -4.48, -4.33, -4.16, -3.98, -3.78,
       -3.57, -3.35, -3.11, -2.85, -2.59, -2.32, -2.03, -1.74, -1.45,
       -1.14, -0.83, -0.52, -0.21,  0.1 ,  0.42,  0.73,  1.04,  1.34,
        1.64,  1.94,  2.22,  2.5 ,  2.77,  3.02,  3.27,  3.5 ,  3.72,
        3.92,  4.11,  4.28,  4.43,  4.57,  4.69,  4.79,  4.87,  4.93,
        4.97,  5.  ,  5.  ,  4.98,  4.95,  4.89,  4.82,  4.72,  4.61,
        4.48,  4.33,  4.16,  3.98,  3.78,  3.57,  3.35,  3.11,  2.85,
        2.59,  2.32,  2.03,  1.74,  1.45,  1.14,  0.83,  0.52,  0.21,
       -0.1 , -0.42, -0.73, -1.04, -1.34, -1.64, -1.94, -2.22, -2.5 ,
       -2.77, -3.02,

In [ ]:
cos3 = np.round(5*np.cos((2*np.pi*time_enter)+(np.pi/5)),2)
cos3

array([ 4.05,  3.85,  3.64,  3.42,  3.19,  2.94,  2.68,  2.41,  2.13,
        1.84,  1.55,  1.24,  0.94,  0.63,  0.31,  0.  , -0.31, -0.63,
       -0.94, -1.24, -1.55, -1.84, -2.13, -2.41, -2.68, -2.94, -3.19,
       -3.42, -3.64, -3.85, -4.05, -4.22, -4.38, -4.52, -4.65, -4.76,
       -4.84, -4.91, -4.96, -4.99, -5.  , -4.99, -4.96, -4.91, -4.84,
       -4.76, -4.65, -4.52, -4.38, -4.22, -4.05, -3.85, -3.64, -3.42,
       -3.19, -2.94, -2.68, -2.41, -2.13, -1.84, -1.55, -1.24, -0.94,
       -0.63, -0.31, -0.  ,  0.31,  0.63,  0.94,  1.24,  1.55,  1.84,
        2.13,  2.41,  2.68,  2.94,  3.19,  3.42,  3.64,  3.85,  4.05,
        4.22,  4.38,  4.52,  4.65,  4.76,  4.84,  4.91,  4.96,  4.99,
        5.  ,  4.99,  4.96,  4.91,  4.84,  4.76,  4.65,  4.52,  4.38,
        4.22,  4.05,  3.85,  3.64,  3.42,  3.19,  2.94,  2.68,  2.41,
        2.13,  1.84,  1.55,  1.24,  0.94,  0.63,  0.31,  0.  , -0.31,
       -0.63, -0.94, -1.24, -1.55, -1.84, -2.13, -2.41, -2.68, -2.94,
       -3.19, -3.42,

In [ ]:
cos4 = np.round(5*np.cos((2*np.pi*time_enter)+(np.pi/4)),2)
cos4

array([ 3.54,  3.31,  3.06,  2.81,  2.55,  2.27,  1.99,  1.69,  1.39,
        1.09,  0.78,  0.47,  0.16, -0.16, -0.47, -0.78, -1.09, -1.39,
       -1.69, -1.99, -2.27, -2.55, -2.81, -3.06, -3.31, -3.54, -3.75,
       -3.95, -4.14, -4.3 , -4.46, -4.59, -4.7 , -4.8 , -4.88, -4.94,
       -4.98, -5.  , -5.  , -4.98, -4.94, -4.88, -4.8 , -4.7 , -4.59,
       -4.46, -4.3 , -4.14, -3.95, -3.75, -3.54, -3.31, -3.06, -2.81,
       -2.55, -2.27, -1.99, -1.69, -1.39, -1.09, -0.78, -0.47, -0.16,
        0.16,  0.47,  0.78,  1.09,  1.39,  1.69,  1.99,  2.27,  2.55,
        2.81,  3.06,  3.31,  3.54,  3.75,  3.95,  4.14,  4.3 ,  4.46,
        4.59,  4.7 ,  4.8 ,  4.88,  4.94,  4.98,  5.  ,  5.  ,  4.98,
        4.94,  4.88,  4.8 ,  4.7 ,  4.59,  4.46,  4.3 ,  4.14,  3.95,
        3.75,  3.54,  3.31,  3.06,  2.81,  2.55,  2.27,  1.99,  1.69,
        1.39,  1.09,  0.78,  0.47,  0.16, -0.16, -0.47, -0.78, -1.09,
       -1.39, -1.69, -1.99, -2.27, -2.55, -2.81, -3.06, -3.31, -3.54,
       -3.75, -3.95,

In [ ]:
cos5 = np.round(5*np.cos((2*np.pi*time_enter)+(np.pi/3)),2)
cos5

array([ 2.5 ,  2.22,  1.94,  1.64,  1.34,  1.04,  0.73,  0.42,  0.1 ,
       -0.21, -0.52, -0.83, -1.14, -1.45, -1.74, -2.03, -2.32, -2.59,
       -2.85, -3.11, -3.35, -3.57, -3.78, -3.98, -4.16, -4.33, -4.48,
       -4.61, -4.72, -4.82, -4.89, -4.95, -4.98, -5.  , -5.  , -4.97,
       -4.93, -4.87, -4.79, -4.69, -4.57, -4.43, -4.28, -4.11, -3.92,
       -3.72, -3.5 , -3.27, -3.02, -2.77, -2.5 , -2.22, -1.94, -1.64,
       -1.34, -1.04, -0.73, -0.42, -0.1 ,  0.21,  0.52,  0.83,  1.14,
        1.45,  1.74,  2.03,  2.32,  2.59,  2.85,  3.11,  3.35,  3.57,
        3.78,  3.98,  4.16,  4.33,  4.48,  4.61,  4.72,  4.82,  4.89,
        4.95,  4.98,  5.  ,  5.  ,  4.97,  4.93,  4.87,  4.79,  4.69,
        4.57,  4.43,  4.28,  4.11,  3.92,  3.72,  3.5 ,  3.27,  3.02,
        2.77,  2.5 ,  2.22,  1.94,  1.64,  1.34,  1.04,  0.73,  0.42,
        0.1 , -0.21, -0.52, -0.83, -1.14, -1.45, -1.74, -2.03, -2.32,
       -2.59, -2.85, -3.11, -3.35, -3.57, -3.78, -3.98, -4.16, -4.33,
       -4.48, -4.61,

In [ ]:
cos6 = np.round(5*np.cos((2*np.pi*time_enter)+(np.pi/2)),2)
cos6

array([-0.  , -0.31, -0.63, -0.94, -1.24, -1.55, -1.84, -2.13, -2.41,
       -2.68, -2.94, -3.19, -3.42, -3.64, -3.85, -4.05, -4.22, -4.38,
       -4.52, -4.65, -4.76, -4.84, -4.91, -4.96, -4.99, -5.  , -4.99,
       -4.96, -4.91, -4.84, -4.76, -4.65, -4.52, -4.38, -4.22, -4.05,
       -3.85, -3.64, -3.42, -3.19, -2.94, -2.68, -2.41, -2.13, -1.84,
       -1.55, -1.24, -0.94, -0.63, -0.31, -0.  ,  0.31,  0.63,  0.94,
        1.24,  1.55,  1.84,  2.13,  2.41,  2.68,  2.94,  3.19,  3.42,
        3.64,  3.85,  4.05,  4.22,  4.38,  4.52,  4.65,  4.76,  4.84,
        4.91,  4.96,  4.99,  5.  ,  4.99,  4.96,  4.91,  4.84,  4.76,
        4.65,  4.52,  4.38,  4.22,  4.05,  3.85,  3.64,  3.42,  3.19,
        2.94,  2.68,  2.41,  2.13,  1.84,  1.55,  1.24,  0.94,  0.63,
        0.31,  0.  , -0.31, -0.63, -0.94, -1.24, -1.55, -1.84, -2.13,
       -2.41, -2.68, -2.94, -3.19, -3.42, -3.64, -3.85, -4.05, -4.22,
       -4.38, -4.52, -4.65, -4.76, -4.84, -4.91, -4.96, -4.99, -5.  ,
       -4.99, -4.96,

In [ ]:
cos7 = np.round(5*np.cos((2*np.pi*wola)+(np.pi)),2)
cos7

array([-5.  , -4.99, -4.96, -4.91, -4.84, -4.76, -4.65, -4.52, -4.38,
       -4.22, -4.05, -3.85, -3.64, -3.42, -3.19, -2.94, -2.68, -2.41,
       -2.13, -1.84, -1.55, -1.24, -0.94, -0.63, -0.31, -0.  ,  0.31,
        0.63,  0.94,  1.24,  1.55,  1.84,  2.13,  2.41,  2.68,  2.94,
        3.19,  3.42,  3.64,  3.85,  4.05,  4.22,  4.38,  4.52,  4.65,
        4.76,  4.84,  4.91,  4.96,  4.99,  5.  ,  4.99,  4.96,  4.91,
        4.84,  4.76,  4.65,  4.52,  4.38,  4.22,  4.05,  3.85,  3.64,
        3.42,  3.19,  2.94,  2.68,  2.41,  2.13,  1.84,  1.55,  1.24,
        0.94,  0.63,  0.31,  0.  , -0.31, -0.63, -0.94, -1.24, -1.55,
       -1.84, -2.13, -2.41, -2.68, -2.94, -3.19, -3.42, -3.64, -3.85,
       -4.05, -4.22, -4.38, -4.52, -4.65, -4.76, -4.84, -4.91, -4.96,
       -4.99, -5.  , -4.99, -4.96, -4.91, -4.84, -4.76, -4.65, -4.52,
       -4.38, -4.22, -4.05, -3.85, -3.64, -3.42, -3.19, -2.94, -2.68,
       -2.41, -2.13, -1.84, -1.55, -1.24, -0.94, -0.63, -0.31, -0.  ,
        0.31,  0.63,

In [ ]:
np.round(time_enter,2)

array([-5.  , -4.99, -4.98, -4.97, -4.96, -4.95, -4.94, -4.93, -4.92,
       -4.91, -4.9 , -4.89, -4.88, -4.87, -4.86, -4.85, -4.84, -4.83,
       -4.82, -4.81, -4.8 , -4.79, -4.78, -4.77, -4.76, -4.75, -4.74,
       -4.73, -4.72, -4.71, -4.7 , -4.69, -4.68, -4.67, -4.66, -4.65,
       -4.64, -4.63, -4.62, -4.61, -4.6 , -4.59, -4.58, -4.57, -4.56,
       -4.55, -4.54, -4.53, -4.52, -4.51, -4.5 , -4.49, -4.48, -4.47,
       -4.46, -4.45, -4.44, -4.43, -4.42, -4.41, -4.4 , -4.39, -4.38,
       -4.37, -4.36, -4.35, -4.34, -4.33, -4.32, -4.31, -4.3 , -4.29,
       -4.28, -4.27, -4.26, -4.25, -4.24, -4.23, -4.22, -4.21, -4.2 ,
       -4.19, -4.18, -4.17, -4.16, -4.15, -4.14, -4.13, -4.12, -4.11,
       -4.1 , -4.09, -4.08, -4.07, -4.06, -4.05, -4.04, -4.03, -4.02,
       -4.01, -4.  , -3.99, -3.98, -3.97, -3.96, -3.95, -3.94, -3.93,
       -3.92, -3.91, -3.9 , -3.89, -3.88, -3.87, -3.86, -3.85, -3.84,
       -3.83, -3.82, -3.81, -3.8 , -3.79, -3.78, -3.77, -3.76, -3.75,
       -3.74, -3.73,

**The relation between the statistical mean and time mean:**

In [ ]:
#the average of the time mean of the sample functions.
h = list()
for i in range(len(the_ENSEMBLE)):
    h.append(st.mean(the_ENSEMBLE[i]))
n= np.sum(h)/len(h)
n

0.0

In [ ]:
#the average of the ensemble mean.
for i in range(len(the_ENSEMBLE[0])):
    h.append(st.mean([row[i] for row in the_ENSEMBLE]))
n = np.sum(h)/len(h)    
n

-2.1168105335454823e-17